## Pipeline of Pando

## Imports

In [1]:
%load_ext rpy2.ipython
work_dir = '../../output'

In [2]:
%%R -i work_dir
options(digits=5, max.print=100)  # Adjust numbers as needed
set.seed(123)
library(Seurat)
library(Matrix)
library(Pando)
library(Signac)
library(BSgenome.Hsapiens.UCSC.hg38)
library(EnsDb.Hsapiens.v86)



    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: SeuratObject
Loading required package: sp
‘SeuratObject’ was built under R 4.1.2 but the current version is
4.3.2; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed
‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed

Attaching package: ‘SeuratObject’

The following object is masked from ‘package:base’:

    intersect


Attaching package: ‘Pando’

The following objects are masked from ‘package:Seurat’:

    GetAssay, VariableFeatures

The following objects are masked from ‘package:SeuratObject’:

    LayerData, VariableFeatures

Loading required package: GenomeInfoDb
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following object is masked from ‘package:SeuratObject’:

    intersect

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

T

## Data formating

In [3]:
%%R 

peaks_count <- readMM(paste0(work_dir, "/scATAC/X_matrix.mtx"))
peaks_count <- t(peaks_count)
annotation_peak <- read.csv(paste0(work_dir, "/scATAC/annotation_peak.csv"), row.names = 1)
annotation_cells_peaks <- read.csv(paste0(work_dir, "/scATAC/annotation_cells.csv"), row.names = 1)

# Filter out entries where seqname is not chr 
filter_indices <- grepl("^chr", annotation_peak$seqname)
annotation_peak <- annotation_peak[filter_indices, ]

# Filter the rows in X
peaks_count <- peaks_count[filter_indices, ]
  
colnames(peaks_count) <- annotation_cells_peaks$obs_id 

rownames(peaks_count) = paste(annotation_peak$seqname, annotation_peak$ranges, sep = "_")


In [4]:
%%R 
rna_counts <- readMM(paste0(work_dir, "/scRNA/X_matrix.mtx"))
rna_counts <- t(rna_counts)
annotation_genes_rna <- read.csv(paste0(work_dir, "/scRNA/annotation_gene.csv"), row.names = 1)
annotation_cells_rna <- read.csv(paste0(work_dir, "/scRNA/annotation_cells.csv"), row.names = 1)

colnames(rna_counts) <- annotation_cells_rna$obs_id 

rownames(rna_counts) = annotation_genes_rna$location 

## Create seurat obj

In [5]:
%%R 

atac_assay <- CreateChromatinAssay(counts = peaks_count, ranges=GRanges(annotation_peak$seqname,
                              IRanges(annotation_peak$ranges)), 
                             colData = DataFrame(annotation_cells), genome = "hg38")
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevelsStyle(annotations) <- "UCSC"
genome(annotations) <- "hg38"
Annotation(atac_assay) <- annotations

  |                                                  | 0 % ~calculating   |++                                                | 4 % ~30s           |++++                                              | 8 % ~27s           |++++++                                            | 12% ~38s           |++++++++                                          | 16% ~36s           |++++++++++                                        | 20% ~34s           |++++++++++++                                      | 24% ~30s           |++++++++++++++                                    | 28% ~28s           |++++++++++++++++                                  | 32% ~26s           |++++++++++++++++++                                | 36% ~24s           |++++++++++++++++++++                              | 40% ~22s           |++++++++++++++++++++++                            | 44% ~21s           |++++++++++++++++++++++++                          | 48% ~19s           |++++++++++++++++++++++++++                        | 52% ~17s 

In addition: There were 24 warnings (use warnings() to see them)


In [6]:
%%R 
seurat_obj <- CreateSeuratObject(counts = rna_counts, project = "pbmc", min.cells = 3, min.features = 200)
seurat_obj[['peaks']] = atac_assay
names(seurat_obj@assays) <- c("RNA", "peaks")

In [7]:
saveRDS(seurat_object, paste0(work_dir, '/pando/seurat_obj.rds'))

NameError: name 'saveRDS' is not defined

## GRN

In [9]:
%%R 
options(digits=5, max.print=100)  # Adjust numbers as needed
set.seed(123)
library(Seurat)
library(Matrix)
library(Pando)
library(Signac)
library(BSgenome.Hsapiens.UCSC.hg38)

data(motifs)

work_dir <- 'output'

seurat_obj = readRDS(paste0(work_dir, '/pando/seurat_obj.rds'))

seurat_obj_v <- Seurat::FindVariableFeatures(seurat_obj, assay='RNA', nfeatures=3000)


seurat_p_obj <- initiate_grn(
    seurat_obj_v,
    rna_assay = 'RNA',
    peak_assay = 'peaks'
)

seurat_p_obj <- find_motifs(
    seurat_p_obj,
    pfm = motifs,
    genome = BSgenome.Hsapiens.UCSC.hg38
)


Finding variable features for layer counts
Calculating gene variances
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Calculating feature variances of standardized and clipped values
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Adding TF info

Building motif matrix
Finding motif positions
Creating Motif object
Warning in CreateMotifObject(data = motif.matrix, positions = motif.positions,  :
  Non-unique motif names supplied, making unique
Loaded glmnet 4.1-8
Selecting candidate regulatory regions near genes
Preparing model input
Fitting models for 2169 target genes
Error in foreach::foreach(i = 1:length(x)) %dopar% { : 
  could not find function "%dopar%"
In addition: Warning message:
Layer 'data' is empty
Error in foreach::foreach(i = 1:length(x)) %dopar% { : 
  c

RInterpreterError: Failed to parse and evaluate line "options(digits=5, max.print=100)  # Adjust numbers as needed\nset.seed(123)\nlibrary(Seurat)\nlibrary(Matrix)\nlibrary(Pando)\nlibrary(Signac)\nlibrary(BSgenome.Hsapiens.UCSC.hg38)\n\nwork_dir <- '../../output'\n\nseurat_obj = readRDS(paste0(work_dir, '/pando/seurat_obj.rds'))\n\nseurat_obj_v <- Seurat::FindVariableFeatures(seurat_obj, assay='RNA', nfeatures=3000)\n\n\nseurat_p_obj <- initiate_grn(\n    seurat_obj_v,\n    rna_assay = 'RNA',\n    peak_assay = 'peaks'\n)\n\nseurat_p_obj <- find_motifs(\n    seurat_p_obj,\n    pfm = motifs,\n    genome = BSgenome.Hsapiens.UCSC.hg38\n)\n\n\n\n\nresults <- infer_grn(\n    seurat_p_obj,\n    parallel = T\n)\n\n\nsaveRDS(results, paste0(work_dir, '/pando/results.rds'))\n".
R error message: 'Error in foreach::foreach(i = 1:length(x)) %dopar% { : \n  could not find function "%dopar%"'
R stdout:
Finding variable features for layer counts
Calculating gene variances
0%   10   20   30   40   50 

In [24]:
%%R 
# seurat_p_obj
saveRDS(seurat_p_obj, paste0(work_dir, '/pando/results.rds'))

: 

In [23]:
%%R 

results <- infer_grn(
    seurat_p_obj
)


# saveRDS(results, paste0(work_dir, '/pando/results.rds'))


  |                                                  | 0 % ~calculating  
Selecting candidate regulatory regions near genes
Preparing model input
Fitting models for 2169 target genes
Error in .subscript.2ary(x, i, j, drop = drop) : 
  logical subscript too long
In addition: Warning message:
Layer 'data' is empty
Error in .subscript.2ary(x, i, j, drop = drop) : 
  logical subscript too long


RInterpreterError: Failed to parse and evaluate line "\nresults <- infer_grn(\n    seurat_p_obj\n)\n\n\n# saveRDS(results, paste0(work_dir, '/pando/results.rds'))\n".
R error message: 'Error in .subscript.2ary(x, i, j, drop = drop) : \n  logical subscript too long'
R stdout:
Selecting candidate regulatory regions near genes
Preparing model input
Fitting models for 2169 target genes
Error in .subscript.2ary(x, i, j, drop = drop) : 
  logical subscript too long
In addition: Warning message:
Layer 'data' is empty

In [3]:
%%R 
seurat_obj = readRDS(paste0(work_dir, '/pando/seurat_obj.rds'))

In [ ]:
%%R 
seurat_obj_v <- Seurat::FindVariableFeatures(seurat_obj, assay='RNA', nfeatures=3000)


In [6]:
%%R 
seurat_p_obj <- initiate_grn(
    seurat_obj_v,
    rna_assay = 'RNA',
    peak_assay = 'peaks'
)

In [ ]:
seurat_p_obj <- find_motifs(
    seurat_p_obj,
    pfm = motifs,
    genome = BSgenome.Hsapiens.UCSC.hg38
)

In [ ]:
results <- infer_grn(
    seurat_p_obj,
    parallel = T
) 
coef(results)

In [ ]:
saveRDS(results, paste0(work_dir, '/pando/results.rds'))

In [1]:
set.seed(123)

# library(Pando)
library(Seurat)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Signac)
library(EnsDb.Hsapiens.v86)

library(Matrix)
#-------- import atac-seq count matrix and metadata and creat assay
X <- readMM("X_matrix.mtx")
X <- t(X)
annotation_peak <- read.csv("annotation_peak.csv", row.names = 1)
annotation_cells <- read.csv("annotation_cells.csv", row.names = 1)

# Filter out entries where seqname is not chr 
filter_indices <- grepl("^chr", annotation_peak$seqname)
annotation_peak_filtered <- annotation_peak[filter_indices, ]

# Filter the rows in X
X_filtered <- X[filter_indices, ]
  
peaks_matrix <- X_filtered
colnames(peaks_matrix) <- annotation_cells$obs_id 

rownames(peaks_matrix) = paste(annotation_peak_filtered$seqname, annotation_peak_filtered$ranges, sep = "_")
atac_assay <- CreateChromatinAssay(counts = peaks_matrix, ranges=GRanges(annotation_peak_filtered$seqname,
                              IRanges(annotation_peak_filtered$ranges)), 
                             colData = DataFrame(annotation_cells), genome = "hg38")
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevelsStyle(annotations) <- "UCSC"
genome(annotations) <- "hg38"
Annotation(atac_assay) <- annotations
#-------- import rna-seq count matrix (with row and col names already assigned ) and create seurat object
rna_short  = readRDS('rna_short.rds')
seurat_object <- CreateSeuratObject(count = rna_short, project = "pbmc", min.cells = 3, min.features = 200)
# add peaks
seurat_object[["peaks"]] = atac_assay

seurat_object

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.1.2 but the current version is
4.3.2; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: GenomeInfoDb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, gre

An object of class Seurat 
157452 features across 7830 samples within 2 assays 
Active assay: RNA (22272 features, 0 variable features)
 1 layer present: counts
 1 other assay present: peaks